In [1]:
import warnings

import lightning.pytorch as pl
import numpy as np
import pandas as pd
import torch
from pytorch_forecasting import TemporalFusionTransformer, TimeSeriesDataSet, CrossEntropy
from pytorch_forecasting.metrics import QuantileLoss
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings('ignore')

In [2]:
def add_features(raw_returns: pd.Series) -> pd.DataFrame:
    features = {}
    hls = [5, 20, 60]

    for hl in hls:
        # Feature 1: EWM-ret
        features[f'ret_{hl}'] = raw_returns.ewm(halflife=hl).mean()
        # Feature 2: log(EWM-DD)
        sq_mean = np.minimum(raw_returns, 0.).pow(2).ewm(halflife=hl).mean()
        dd = np.sqrt(sq_mean)
        features[f'dd-log_{hl}'] = np.log(dd)
        # Feature 3: EWM-Sortino-ratio = EWM-ret/EWM-DD
        features[f'sortino_{hl}'] = features[f'ret_{hl}'].div(dd)

    return pd.DataFrame(features)


def identify_regimes(dataframe: pd.DataFrame) -> pd.DataFrame:
    # Create regime-specific features
    scaled_features = StandardScaler().fit_transform(dataframe)

    # Apply Gaussian Mixture Model
    gmm = GaussianMixture(n_components=2, random_state=42)
    regimes = gmm.fit_predict(scaled_features)

    # Ensure we have the right length
    full_regimes = np.full(len(dataframe), -1)
    start_idx = len(dataframe) - len(regimes)
    full_regimes[start_idx:] = regimes

    return full_regimes

training_cutoff = 5808
eval_cutoff = 11010

df = pd.read_csv('sp500.csv', parse_dates=['date'])
log_returns = df['returns']

# Compute relevant features and identify regime labels
data = add_features(log_returns)
data['returns'] = log_returns
data['regime'] = identify_regimes(data).astype(str)
data['date'] = df['date']

data.to_csv('sp500_regimes.csv', index=False)

# Add required columns for TimeSeriesDataSet
data['fin_type'] = 'sp500'
data['time_idx'] = data.index

In [3]:
max_prediction_length = 30
max_encoder_length = 32
batch_size = 128
hidden_size = 16
max_epochs = 50

# Manually select accelerator to avoid MPS due to current performances issues
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
workers = 0

training = TimeSeriesDataSet(
    data[:training_cutoff],
    time_idx='time_idx',
    target='regime',
    group_ids=['fin_type'],
    min_encoder_length=max_encoder_length // 2,
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=['fin_type'],
    time_varying_known_reals=['time_idx', 'returns'],
    time_varying_unknown_categoricals=['regime'],
    time_varying_unknown_reals=[
        'ret_5',
        'dd-log_5',
        'sortino_5',
        'ret_20',
        'dd-log_20',
        'sortino_20',
        'ret_60',
        'dd-log_60',
        'sortino_60',
    ],
    add_encoder_length=True,
)

train_dataloader = training.to_dataloader(
    train=True, batch_size=batch_size, num_workers=workers
)

evaluation = TimeSeriesDataSet.from_dataset(
    training, data, predict=True, stop_randomization=True
)

eval_dataloader = evaluation.to_dataloader(
    train=False, batch_size=batch_size * 10, num_workers=workers
)

In [4]:
pl.seed_everything(42)

trainer = pl.Trainer(accelerator=accelerator, gradient_clip_val=0.1)

tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,  # not meaningful for finding the learning rate
    hidden_size=hidden_size,
    attention_head_size=2,
    dropout=0.1,
    hidden_continuous_size=8,
    loss=QuantileLoss(),
    optimizer='ranger',
)
print(f'Number of parameters in network: {tft.size() / 1e3:.1f}k')

Seed set to 42
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Number of parameters in network: 23.4k


In [5]:
# find optimal learning rate
from lightning.pytorch.tuner import Tuner

res = Tuner(trainer).lr_find(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=eval_dataloader,
    max_lr=10.0,
    min_lr=1e-6,
)

print(f'suggested learning rate: {res.suggestion()}')
fig = res.plot(show=True, suggest=True)
fig.show()

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/paul/anaconda3/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/paul/anaconda3/lib/python3.12/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/paul/anaconda3/lib/python3.12/site-packages/pytorch_forecasting/__init__.py", line 7, in <module>
    from pytorch_forecasting.data import (
  File "/Users/paul/anaconda3/lib/python3.12/site-packages/pytorch_forecasting/data/__init__.py", line 8, in <module>
    from pytorch_forecasting.data.encoders import (
  File "/Users/paul/anaconda3/lib/python3.12/site-packages/pytorch_forecasting/data/encoders.py", line 10, in <module>
    import pandas as pd
  File "/Users/paul/anaconda3/lib/python3.12/site-packages/pandas/__init__.py", line 142, in <

NameError: name 'exit' is not defined

In [6]:
trainer = pl.Trainer(
    max_epochs=max_epochs,
    accelerator=accelerator,
    enable_model_summary=True,
    gradient_clip_val=0.1,
    limit_train_batches=50,
)

tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.015,
    hidden_size=hidden_size,
    attention_head_size=2,
    dropout=0.1,
    hidden_continuous_size=8,
    loss=CrossEntropy(),
    log_interval=10,
    optimizer='ranger',
    reduce_on_plateau_patience=4,
)
print(f'Number of parameters in network: {tft.size() / 1e3:.1f}k')

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Number of parameters in network: 23.3k


In [7]:
trainer.fit(tft, train_dataloaders=train_dataloader)


   | Name                               | Type                            | Params | Mode 
------------------------------------------------------------------------------------------------
0  | loss                               | CrossEntropy                    | 0      | train
1  | logging_metrics                    | ModuleList                      | 0      | train
2  | input_embeddings                   | MultiEmbedding                  | 3      | train
3  | prescalers                         | ModuleDict                      | 192    | train
4  | static_variable_selection          | VariableSelectionNetwork        | 624    | train
5  | encoder_variable_selection         | VariableSelectionNetwork        | 7.7 K  | train
6  | decoder_variable_selection         | VariableSelectionNetwork        | 1.2 K  | train
7  | static_context_variable_selection  | GatedResidualNetwork            | 1.1 K  | train
8  | static_context_initial_hidden_lstm | GatedResidualNetwork            | 1.1 K  

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.


In [8]:
raw_predictions = []

for t in range(training_cutoff, eval_cutoff, max_prediction_length):
    evaluation = TimeSeriesDataSet.from_dataset(
        training, data[:t], predict=True, stop_randomization=True
    )

    eval_dataloader = evaluation.to_dataloader(
        train=False, batch_size=batch_size * 10, num_workers=workers
    )

    y_pred = tft.predict(eval_dataloader, return_y=True, trainer_kwargs=dict(accelerator=accelerator)).y[0]
    y_pred = y_pred.flatten()
    raw_predictions.append(y_pred.mean(dtype=float).item())

raw_predictions = np.array(raw_predictions)

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True 

In [14]:
import seaborn as sns
from evaluation import plot_regimes, evaluate_strategy

sns.set_theme(style='whitegrid', context='paper')

predictions = np.where(raw_predictions > 0.5, 1, 0).repeat(max_prediction_length)

eval_dates = df['date'].iloc[training_cutoff:eval_cutoff].to_numpy()
eval_prices = df['close'].iloc[training_cutoff:eval_cutoff]
eval_returns = log_returns.iloc[training_cutoff:eval_cutoff]

plot_regimes(eval_dates, eval_prices, predictions[-eval_prices.size:], fig_size=(10, 4), save_path='tft')

evaluate_strategy(eval_returns, predictions[-eval_prices.size:], 1)
evaluate_strategy(eval_returns, predictions[-eval_prices.size:], 0.7)
evaluate_strategy(eval_returns, predictions[-eval_prices.size:], 0.3)
evaluate_strategy(eval_returns, predictions[-eval_prices.size:], 0)

Statistics for 100% hedged:
 CVaR (95.0%): -1.85%
 MDD: -33.9%
 Annual Log Return: 6.46%
 Annual Volatility: 11.9%
 Sharpe Ratio: 0.544
Statistics for 70.0% hedged:
 CVaR (95.0%): -1.92%
 MDD: -33.9%
 Annual Log Return: 6.85%
 Annual Volatility: 12.7%
 Sharpe Ratio: 0.539
Statistics for 30.0% hedged:
 CVaR (95.0%): -2.42%
 MDD: -45.1%
 Annual Log Return: 7.37%
 Annual Volatility: 15.9%
 Sharpe Ratio: 0.465
Statistics for 0% hedged:
 CVaR (95.0%): -3.00%
 MDD: -56.8%
 Annual Log Return: 7.77%
 Annual Volatility: 19.1%
 Sharpe Ratio: 0.406
